<a href="https://colab.research.google.com/github/Vaycold/pytorch_tutorial/blob/main/%2304.%EC%8B%A0%EA%B2%BD%EB%A7%9D%20%EB%AA%A8%EB%8D%B8%20%EA%B5%AC%EC%84%B1%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 신경망 모델 구성하기

    : 신경망은 데이터에 대한 연산을 수행하는 계층/ 모듈로 구성
    : torch.nn namespace - 신경망을 구성하는 데 필요한 모든 구성요소 제공
    : PyTorch의 모든 모듈은 nn.Module의 하위 클래스
    : 신경망은 다른 모듈로 구성된 모듈
    

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


### 클래스 정의

    : 신경망 모델을 nn.Module의 하위 클래스로 정의
    : __init__ 에서 신경망 계층들을 초기화
    : nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현


In [4]:
class NeuralNetwork(nn.Module) :
    def __init__(self) :
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.ReLU()
        )
    
    def forward(self, x) :
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [6]:
x = torch.rand(1,28,28, device = device)
logits = model(x)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print('predicted class : ', y_pred)

predicted class :  tensor([7])


In [8]:
# 모델 계층(Layer)
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
# nn.Flatten : 28x28의 2D 이미지를 784픽셀 값을 갖는 연속된 배열로 변환 
# 이 때 미니배치의 차원은 유지
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
# nn.Linear
# 선형 계층은 저장된 가중치와 편향을 사용하여 입력에 선형 변환을 적용하는 모듈
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [15]:
# nn.ReLU
# 비선형 활성화는 모델의 입력과 출력사이에 복잡한 관계를 만듬
# 비선형 활성화는 선형 변환 후에 적용되어 비선형성을 도입하고 신경망이 다양한 현상을 학습할 수 있도록 도움

print( 'Before ReLU')
print(hidden1)
print('*'*100)
print('After ReLU')
hidden1 = nn.ReLU()(hidden1)
print(hidden1)

Before ReLU
tensor([[-0.1771, -0.2340,  0.1187,  0.0797,  0.3301, -0.1149, -0.3920,  0.3216,
         -0.6674,  0.7789,  0.6370, -0.1812, -0.2905, -0.1899, -0.0121, -0.2047,
         -0.4074,  0.2750,  0.6261,  0.2650],
        [-0.2580, -0.1416,  0.0415, -0.0641,  0.2165,  0.1917, -0.4925, -0.2071,
         -0.3075,  0.4703,  0.6017, -0.4164, -0.0620, -0.4173,  0.0444, -0.5167,
         -0.3085,  0.1961,  0.3941,  0.2415],
        [-0.2436, -0.3379,  0.0588,  0.1698,  0.2068,  0.6604, -0.5532,  0.0314,
         -0.3529,  0.6564,  0.7165, -0.2779, -0.0416, -0.2930,  0.4511,  0.0047,
         -0.4948,  0.4627,  0.2174,  0.4278]], grad_fn=<AddmmBackward>)
****************************************************************************************************
After ReLU
tensor([[0.0000, 0.0000, 0.1187, 0.0797, 0.3301, 0.0000, 0.0000, 0.3216, 0.0000,
         0.7789, 0.6370, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2750,
         0.6261, 0.2650],
        [0.0000, 0.0000, 0.0415, 0.000

In [16]:
# nn.Sequential
# 순서를 갖는 모듈의 컨테이너
# 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됨
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)

) 
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [17]:
# nn.Softmax
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [22]:
# 모델의 매개변수
# 신경망 내부의 많은 계층들은 매개변수화(parameterize)됨
# 즉 학습 중 최적화되는 가중치와 편향과 연관지어짐
# 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게됨

print('model structure', model, '\n')
for name, param in model.named_parameters() :
    print('layer : {} | Size : {} | Values : {} \n'.format(name,param.size(),param[:2]))

model structure NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 

layer : linear_relu_stack.0.weight | Size : torch.Size([512, 784]) | Values : tensor([[ 0.0206,  0.0119,  0.0346,  ..., -0.0074,  0.0331,  0.0159],
        [-0.0357,  0.0002,  0.0245,  ...,  0.0169, -0.0122, -0.0177]],
       grad_fn=<SliceBackward>) 

layer : linear_relu_stack.0.bias | Size : torch.Size([512]) | Values : tensor([-0.0146,  0.0325], grad_fn=<SliceBackward>) 

layer : linear_relu_stack.2.weight | Size : torch.Size([512, 512]) | Values : tensor([[-0.0337,  0.0200,  0.0283,  ...,  0.0321, -0.0237, -0.0098],
        [ 0.0182, -0.0225, -0.0054,  ...,  0.0122, -0.0331,  0.0124]],
       grad_fn=<SliceBackward>) 

layer : linear_r